<a href="https://colab.research.google.com/github/nidhin-koshy/ML_SP_GAN/blob/master/ML_for_SP_GAN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
if(not os.path.exists("ML_SP_GAN")):
  ! git clone https://github.com/nidhin-koshy/ML_SP_GAN.git

Cloning into 'ML_SP_GAN'...
remote: Enumerating objects: 61, done.
remote: Total 61 (delta 0), reused 0 (delta 0), pack-reused 61
Unpacking objects: 100% (61/61), done.


In [2]:
! cd ML_SP_GAN && git pull 

Already up to date.


In [0]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import normalize

import sys
sys.path.append('ML_SP_GAN')
from myFunctions import features_1 # myFunctions.py is available as part of the the Github repository.
from myFunctions import features_2

#Data set specific variables
freq = 125
dataleninsec = 8
dataoffsetinsec = 2 
numsampleswin = freq*dataleninsec

data_dir_path = './ML_SP_GAN/ppg_test_data_csv/Training_data/' #'/content/gdrive/My Drive/ML_SP_PESIT/ml-course-code-may14/ml-course-code/ppg_test_data_csv/Training_data/'   #'../ppg_test_data_csv/Training_data/'

train_set_02 = ['DATA_02_TYPE02.csv', 'DATA_04_TYPE02.csv', 'DATA_06_TYPE02.csv', 'DATA_08_TYPE02.csv', 'DATA_10_TYPE02.csv', 'DATA_12_TYPE02.csv', 'DATA_03_TYPE02.csv', 'DATA_05_TYPE02.csv', 'DATA_07_TYPE02.csv', 'DATA_09_TYPE02.csv', 'DATA_11_TYPE02.csv']
#
train_set_02_BPM = ['DATA_02_TYPE02_BPMtrace.csv', 'DATA_04_TYPE02_BPMtrace.csv', 'DATA_06_TYPE02_BPMtrace.csv', 'DATA_08_TYPE02_BPMtrace.csv', 'DATA_10_TYPE02_BPMtrace.csv', 'DATA_12_TYPE02_BPMtrace.csv', 'DATA_03_TYPE02_BPMtrace.csv', 'DATA_05_TYPE02_BPMtrace.csv', 'DATA_07_TYPE02_BPMtrace.csv', 'DATA_09_TYPE02_BPMtrace.csv', 'DATA_11_TYPE02_BPMtrace.csv']

#train_set_02 = ['DATA_04_TYPE02.csv']
#train_set_02_BPM = ['DATA_04_TYPE02_BPMtrace.csv']

for file_ind in np.arange(0,len(train_set_02)):
   #print train_set_02[file_ind]
   #print train_set_02_BPM[file_ind]

   filepath = data_dir_path+train_set_02[file_ind]
   
   #Load training data from a file
   data = np.loadtxt(open(filepath, "rb"), delimiter=",")
   
   
   datalen = data.shape[0]  #Total length of data
   #Indexes to split a single array into matrix. A single BPM is computed over an interval of 8s (1000 samples). 
   #The next number over another 8s window starting at an offset of 2s (500 samples) from previous window

   index_arr = np.arange(0, datalen-numsampleswin+1, freq*dataoffsetinsec)  
   #print datalen #print index_arr.shape[0] #print index_arr

   datax=np.zeros(shape=(index_arr.shape[0], numsampleswin, 6),dtype=np.float)
   datax_pd=np.zeros(shape=(index_arr.shape[0], numsampleswin, 6),dtype=np.float)
   temp = pd.read_csv(filepath)
   temp.head()
   for j in np.arange(0,6):
    for i in range(index_arr.shape[0]) : datax[i,:,j] = data[index_arr[i]:numsampleswin+index_arr[i], j]
        
   
   #For training we are ignoring the ECG signal as this signal is not available for test data. ECG output
   #is available as _BPM.csv dataset
   
   #Use datax1 to compute features to be used in training
   
   datax1=np.zeros(shape=(index_arr.shape[0], numsampleswin, 5),dtype=np.float)
   datax1=datax[:,:,1:6]
   #j = 1 
   #err = []
   #for i in np.arange(0,148):
   #  #err.append(sum(datax[i,:,j] - np.transpose(data[250*i:1000+250*i,j])))
   #  err.append(sum(datax1[i,:,j] - np.transpose(data[250*i:1000+250*i,j+1])))
   #print ("Testing data matrix creation Error: ", sum(err))
   
   
   #X = np.column_stack((np.ones([index_arr.shape[0],1], dtype=float), features_2(datax1)))
   X = features_2(datax1)
   X = normalize(X, norm='l2', axis=0, return_norm=False)
   
   if file_ind == 0: 
      X_aggregate = np.copy(X)
   else:
      X_aggregate = np.vstack((X_aggregate, X))
   #print X.shape
   #print X_aggregate.shape
   #
   
   outfilepath = data_dir_path+train_set_02_BPM[file_ind]
   #Load output training data from a file
   outdata = np.loadtxt(open(outfilepath, "rb"), delimiter=",")

   #print outdata.shape[0]
   if file_ind == 0: 
      Y_aggregate = np.copy(outdata)
   else:
      #Y_aggregate = np.vstack((Y_aggregate, outdata))
      Y_aggregate = np.hstack((Y_aggregate, outdata))


#Split the data into train and cross-validate
npts = 143 
print (Y_aggregate.shape)
print (X_aggregate.shape)
X_train = X_aggregate[:-npts, :]
X_test = X_aggregate[-npts:, :]
Y_train = Y_aggregate[:-npts]
Y_test = Y_aggregate[-npts:]


# Create linear regression object
regr = linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X_train, Y_train)
pred_Y = regr.predict(X_test)


#print('Variance score: %.2f' % r2_score(pred_Y, Y_test))

#for i in np.arange(0,Y_test.shape[0]):
#   print np.column_stack((pred_Y[i],Y_test[i]))

# Create linear regression object
#regr1 = linear_model.Ridge(alpha=10.0)
regr1 = linear_model.RidgeCV(alphas=np.logspace(-3, 5, 9),fit_intercept=True,cv=2)

# Train the model using the training sets
regr1.fit(X_train, Y_train)
pred_Y_ridge = regr1.predict(X_test)


# Create linear regression object
#regr2 = linear_model.Lasso(alpha=0.35)
regr2 = linear_model.LassoCV(alphas=np.logspace(-3, 5, 9),cv=2)

# Train the model using the training sets
regr2.fit(X_train, Y_train)
pred_Y_lasso = regr2.predict(X_test)

# The coefficients
print('Coefficients - Linear Regression: \n', regr.coef_)
print('Coefficients - Ridge: \n', regr1.coef_)
print('Coefficients - Lasso: \n', regr2.coef_)

print('Intercept - Linear Regression: \n', regr.intercept_)
print('Intercept - Ridge: \n', regr1.intercept_)
print('Intercept - Lasso: \n', regr2.intercept_)

print("Alphas from Cross Validation: ridge_alpha: %f, lasso_alpha: %f" %(regr1.alpha_, regr2.alpha_))

#for i in np.arange(0,Y_test.shape[0]):
#   print np.column_stack((pred_Y[i],pred_Y_ridge[i], pred_Y_lasso[i], Y_test[i]))

print("MSE, Validation; Linear Reg : %.2f, Ridge : %.2f, Lasso : %.2f"
      % (mean_squared_error(pred_Y, Y_test), mean_squared_error(pred_Y_ridge, Y_test), mean_squared_error(pred_Y_lasso, Y_test)))
############################################################
# Predicting Test Data
############################################################

test_set = ['TEST_S01_T01.csv', 'TEST_S02_T02.csv', 'TEST_S04_T02.csv', 'TEST_S06_T01.csv', 'TEST_S07_T02.csv', 'TEST_S02_T01.csv', 'TEST_S03_T02.csv', 'TEST_S05_T02.csv', 'TEST_S06_T02.csv', 'TEST_S08_T01.csv']
true_test_set = ['True_S01_T01.csv', 'True_S02_T02.csv', 'True_S04_T02.csv', 'True_S06_T01.csv', 'True_S07_T02.csv', 'True_S02_T01.csv', 'True_S03_T02.csv', 'True_S05_T02.csv', 'True_S06_T02.csv', 'True_S08_T01.csv']
#test_set = ['TEST_S01_T01.csv', 'TEST_S02_T02.csv', 'TEST_S04_T02.csv', 'TEST_S06_T01.csv', 'TEST_S07_T02.csv']
#true_test_set = ['True_S01_T01.csv', 'True_S02_T02.csv', 'True_S04_T02.csv', 'True_S06_T01.csv', 'True_S07_T02.csv']
#test_set = ['TEST_S03_T02.csv']
#true_test_set = ['True_S03_T02.csv']
test_dir_path =  './ML_SP_GAN/ppg_test_data_csv/TestData/' #'/content/gdrive/My Drive/ML_SP_PESIT/ml-course-code-may14/ml-course-code/ppg_test_data_csv/TestData/' #'../ppg_test_data_csv/TestData/'
true_test_dir_path = './ML_SP_GAN/ppg_test_data_csv/TrueBPM/' #'/content/gdrive/My Drive/ML_SP_PESIT/ml-course-code-may14/ml-course-code/ppg_test_data_csv/TrueBPM/' # '../ppg_test_data_csv/TrueBPM/'
for file_ind in np.arange(0,len(test_set)):
   print (test_set[file_ind])
   #print true_test_set[file_ind]
   filepath = test_dir_path+test_set[file_ind]
   
   #Load training data from a file
   data = np.loadtxt(open(filepath, "rb"), delimiter=",")
   #print data.shape
   
   datalen = data.shape[0]  #Total length of data
   #Indexes to split a single array into matrix. A single BPM is computed over an interval of 8s (1000 samples). 
   #The next number over another 8s window starting at an offset of 2s (500 samples) from previous window

   index_arr = np.arange(0, datalen-numsampleswin+1, freq*dataoffsetinsec)  
   datax=np.zeros(shape=(index_arr.shape[0], numsampleswin, 5),dtype=np.float)
   for j in np.arange(0,5):
    for i in range(index_arr.shape[0]): datax[i,:,j] = data[index_arr[i]:numsampleswin+index_arr[i], j]

   #print datax.shape
   
   #X = np.column_stack((np.ones([index_arr.shape[0],1], dtype=float), features_2(datax)))
   X = features_2(datax)
   X = normalize(X, norm='l2', axis=0, return_norm=False)

   outfilepath = true_test_dir_path+true_test_set[file_ind]
   #Load output training data from a file
   trueY = np.loadtxt(open(outfilepath, "rb"), delimiter=",")
   #print trueY.shape

   pred_test_Y       = regr.predict(X)
   pred_test_Y_ridge = regr1.predict(X)
   pred_test_Y_lasso = regr2.predict(X)
   print("MSE, Test Linear Reg : %.2f, Ridge : %.2f, Lasso : %.2f"
      % (mean_squared_error(pred_test_Y, trueY), mean_squared_error(pred_test_Y_ridge, trueY), mean_squared_error(pred_test_Y_lasso, trueY)))
   
   #for i in np.arange(0,trueY.shape[0]):
   #   print np.column_stack((pred_test_Y[i], pred_test_Y_ridge[i], pred_test_Y_lasso[i], trueY[i]))

(1620,)
(1620, 75)
Coefficients - Linear Regression: 
 [-2.79436588e+00  2.74522070e+01 -4.37423942e+01  2.43599477e+03
 -2.59312106e+02  8.43613236e+00  2.81170581e+01 -2.05211973e+01
 -3.79984413e+03  2.82870768e+02 -6.56552417e-01 -9.11513251e+00
 -1.18842924e+01  1.81408466e+03 -2.42045836e+02  8.65821074e+01
  7.54466588e+01 -5.58168588e+01  3.90076353e+02  2.19031872e+02
 -6.83245497e+00  1.12498331e+01  1.05030007e+01 -1.40421488e+01
  1.72360164e+01  1.14881427e+01 -4.85902826e+00  2.57949172e+00
  6.28007297e+00 -2.82248783e-01  9.56243749e+00 -1.27648182e+01
  6.07771785e+00  1.09764605e+01  2.28338693e+00 -1.51406653e+02
 -1.10054143e+02 -1.89369247e+02 -5.71403411e+01  5.88953293e+01
 -9.11506153e+00  1.71570333e+01 -1.73646416e+01 -5.28890109e+00
 -3.29307716e+00  3.66923869e+00 -3.28453035e+00 -2.49722481e+00
  1.86331460e+01 -6.04306173e+00  1.27017668e+01 -5.04752832e+00
  1.69752805e+01  2.10239848e+01 -7.27026783e+00  1.08441688e+02
  3.08244783e+00 -1.13150235e+01 -1

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 107.38554202401428, tolerance: 36.746030311966926
  tol, rng, random, positive)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/coordinate_descent.py:471: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 116.14150917960797, tolerance: 44.99337475515373
  tol, rng, random, positive)


MSE, Test Linear Reg : 221046.99, Ridge : 3058.76, Lasso : 3340.76
TEST_S02_T02.csv
MSE, Test Linear Reg : 486875.16, Ridge : 463.43, Lasso : 395.41
TEST_S04_T02.csv
MSE, Test Linear Reg : 1039.99, Ridge : 490.43, Lasso : 355.87
TEST_S06_T01.csv
MSE, Test Linear Reg : 101314.92, Ridge : 1828.05, Lasso : 1803.44
TEST_S07_T02.csv
MSE, Test Linear Reg : 547852.20, Ridge : 80.90, Lasso : 81.02
TEST_S02_T01.csv
MSE, Test Linear Reg : 151095.83, Ridge : 3006.78, Lasso : 3105.73
TEST_S03_T02.csv
MSE, Test Linear Reg : 504579.99, Ridge : 1194.47, Lasso : 973.98
TEST_S05_T02.csv
MSE, Test Linear Reg : 218491.49, Ridge : 143.94, Lasso : 87.40
TEST_S06_T02.csv
MSE, Test Linear Reg : 452348.70, Ridge : 328.33, Lasso : 240.71
TEST_S08_T01.csv
MSE, Test Linear Reg : 537943.61, Ridge : 2404.16, Lasso : 2336.49
